In [2]:
#pip installs
%pip install opencv-python-headless transformers torch
%pip install faiss-cpu



Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (29.3 MB)
  Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached faiss_cpu-1.9.0.post1-cp310-cp310-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (3.6 MB)
  Using cached numpy-2.2.0-cp310-cp310-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (14.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip uninstall numpy -y

Found existing installation: numpy 2.2.0
Uninstalling numpy-2.2.0:
  Successfully uninstalled numpy-2.2.0
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install 'accelerate>=0.26.0'

Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install qwen-vl-utils

Defaulting to user installation because normal site-packages is not writeable
  Using cached qwen_vl_utils-0.0.8-py3-none-any.whl (5.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, AutoModelForCausalLM, AutoTokenizer, AutoModel
from pathlib import Path

from qwen_vl_utils import process_vision_info 
# from sentence_transformers import SentenceTransformer, util
import faiss
from pathlib import Path
from transformers import pipeline



/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-10 14:49:09.492178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-10 14:49:09.500588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-10 14:49:09.504158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def extract_frames(video_path, frames_dir="./", frame_rate=0.2):
    """Extract frames from a video at a specific rate and save to a directory."""
    video_name = Path(video_path).stem
    video_frames_dir = Path(frames_dir) / video_name
    video_frames_dir.mkdir(parents=True, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps / frame_rate)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % interval == 0:
            frame_path = video_frames_dir / f"{saved_count:06d}.jpg"
            cv2.imwrite(str(frame_path), frame)
            saved_count += 1
        frame_count += 1

    cap.release()
    print(f"Extracted {saved_count} frames from {video_path} to {video_frames_dir}")
    return video_frames_dir, saved_count

In [ ]:

def generate_captions(video_frames_dir):
    """Generate captions for video chunks using the Qwen model in batches of 10 frames."""
    captions = []

    # Sort the frames directory by file name to ensure proper order
    frame_paths = sorted(video_frames_dir.glob("*.jpg"))  # Adjust if your frames are stored with a different extension

    # Define your prompt (as given)
    prompt = (
        "Describe the scene at the self-checkout station, focusing on the customer's actions, especially with their hands. "
        "Look for any signs of fraud, such as:\n"
        "- Not scanning items.\n"
        "- Concealing items.\n"
        "- Tampering with the checkout system.\n"
        "- Suspicious interactions or behaviors.\n\n"
        "Highlight any actions that suggest theft or fraud.\n\n"
        "Description:"
    )

    # Process frames in batches of 10
    for i in range(0, len(frame_paths), 10):
        # Get the next batch of up to 10 frames
        batch_paths = frame_paths[i:i + 10]

        # Create the message structure for model input
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": [str(path) for path in batch_paths]},  # Pass image paths directly
                    {"type": "text", "text": prompt},
                ]
            }
        ]

        # Process frames and prepare inputs for the model
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        image_inputs, video_inputs = process_vision_info(messages)

        # Prepare the input for the model
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt"
        ).to("cuda")

        # Inference: Generate output from the model
        generated_ids = model.generate(**inputs, max_new_tokens=512)  # Customize max length as needed

        # Trim the generated output to remove the input prompt
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        # Decode the output into human-readable captions
        output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)

        # Print the generated caption for the batch
        print(f"Generated caption for batch starting with {batch_paths[0].stem}: {output_text[0]}")

        # Append the caption and its corresponding batch identifier (starting frame)
        captions.append((batch_paths[0].stem, output_text[0]))

    return captions


In [5]:
video_path = "./video/test.mp4"

In [ ]:
# Initialize the Qwen model and processor for video captioning
model_name = "Qwen/Qwen2-VL-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# model = Qwen2VLForConditionalGeneration.from_pretrained(model_name).to(device)
model = Qwen2VLForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.bfloat16, attn_implementation="flash_attention")
processor = AutoProcessor.from_pretrained(model_name)

embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
embedding_model = AutoModel.from_pretrained(embedding_model_name).to(device)

llm_model_name = "meta-llama/Llama-2-13b-chat-hf"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name).to(device)

Loading checkpoint shards: 100%|██████████| 5/5 [00:01<00:00,  4.62it/s]
/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.16it/s]


In [9]:
model

Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Seq

In [11]:
def encode_texts(texts, tokenizer, model):
    """
    Encode a list of texts into embeddings using a Hugging Face model.

    Args:
        texts (list of str): List of input texts to encode.
        tokenizer: Hugging Face tokenizer.
        model: Hugging Face model.

    Returns:
        numpy.ndarray: Embeddings for the input texts.
    """
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Get embeddings from the model
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Use mean pooling
    return embeddings


In [12]:
def create_faiss_index(captions):
    """
    Create a FAISS index to store and search embeddings for caption similarity.

    Args:
        captions (list of tuples): Each tuple contains (frame_id, caption_text), where 'caption_text' is a caption
                                   string to be embedded and indexed.

    Returns:
        index (faiss.IndexFlatL2): A FAISS index with stored embeddings that allows similarity searches.
        captions (list of tuples): The original captions list, which maintains the frame and caption structure.
    """
    # Extract the caption text from each tuple (frame_id, caption_text) for embedding
    captions_text = [caption[1] for caption in captions]

    # Generate embeddings for each caption using the Hugging Face embedding model
    embeddings = encode_texts(captions_text, embedding_tokenizer, embedding_model)

    # Initialize a FAISS index with L2 (Euclidean) distance metric using the dimensionality of the embeddings
    index = faiss.IndexFlatL2(embeddings.shape[1])

    # Add the embeddings to the FAISS index, enabling fast similarity search on these vectors
    index.add(embeddings)

    # Return both the FAISS index and the original captions list for later retrieval and reference
    return index, captions


In [13]:
def refine_captions(index, captions, k=5):
    """
    Refine captions by finding the most semantically similar captions within a neighborhood using FAISS.

    Args:
        index (faiss.IndexFlatL2): A FAISS index containing embeddings for similarity search.
        captions (list of tuples): Original captions with structure (frame_id, caption_text).
        k (int): Number of nearest neighbors to consider.

    Returns:
        list of tuples: Refined captions with the same structure as the input.
    """
    # Extract the caption text from each tuple for embedding
    captions_text = [caption[1] for caption in captions]

    # Generate embeddings for the captions using the Hugging Face embedding model
    embeddings = encode_texts(captions_text, embedding_tokenizer, embedding_model)

    # Search for the k nearest neighbors in the FAISS index
    _, neighbors = index.search(embeddings, k)

    # Refine captions based on the most frequent neighbor caption
    refined_captions = []
    for idx, neighbor_indices in enumerate(neighbors):
        neighbor_texts = [captions[neighbor][1] for neighbor in neighbor_indices]
        most_frequent_caption = max(set(neighbor_texts), key=neighbor_texts.count)
        refined_captions.append((captions[idx][0], most_frequent_caption))
    
    return refined_captions


In [14]:
import re

def anomaly_score(summaries):
    """Assign anomaly scores to each summary using LLM based on predefined behavior patterns."""
    scores = []

    for summary in summaries:
        # System prompt to guide the model
        system_prompt = (
            "You are an expert in detecting suspicious behavior in retail environments like shopping malls.\n"
            "Your task is to evaluate descriptions of customer actions and identify potential signs of theft or suspicious behavior. "
            "Please follow these steps:"
            "1. Explain your thought process in detail, outlining any behaviors or actions that led you to conclude whether theft or suspicious behavior might be happening. DO NOT MENTION ANY SCORE OR NUMBER IN YOUR EXPLANATION"
            "2. Based on your reasoning, provide a floating point number that lies between 0 and 1, where: "
            "   - 0 means no signs of theft, and "
            "   - 1 means a high likelihood of theft."
        )

        # Create the full prompt with system instruction
        prompt = (
            system_prompt + "\n"
            "Scene Description:" + summary[1] + "\n\n"
            "THEFT SCORE:"
        )

        # Encode the prompt for the T5 model
        inputs = llm_tokenizer(prompt, return_tensors="pt").to(device)

        # Generate a response from the model
        output = llm_model.generate(
            **inputs,
            max_length=1000,  # Ensure the model doesn't generate overly long outputs
            eos_token_id=llm_tokenizer.eos_token_id,
        )

        # Decode the response
        response = llm_tokenizer.decode(output[0], skip_special_tokens=True)

        # Remove the initial prompt from the result
        if "THEFT SCORE:" in response:
            response = response.split("THEFT SCORE:")[1].strip()
        else:
            response = response.strip()

        print("Response: ", response)

        # Use regex to extract a floating point number between 0 and 1, or integers like 0 or 1
        match = re.search(r'\b0?(\.\d+)?\b', response)
        if match:
            score = float(match.group())
            print("Extracted score is: ", score)
        else:
            score = None
            print("No score found in response")

        # Append the score to the list
        scores.append(score)

    return scores


In [15]:
def is_anomoly(scores, threshold=0.7):
    # Check if any frame has a score above the threshold
    is_anomalous = any(score >= threshold for score in scores)

    # Return "Yes" if any anomaly is detected, otherwise "No"
    return "Yes" if is_anomalous else "No"

In [16]:
def run_eval(video_dir, output_csv="evaluation_results.csv", threshold=0.6):
    """
    Evaluate the LAVAD flow for multiple videos and generate a CSV result.
    If the CSV file exists, it appends new results; otherwise, it creates a new file.

    Args:
        video_dir (str): Directory containing video files to process.
        output_csv (str): Path to save the evaluation results as a CSV file.
        threshold (float): Anomaly detection threshold.

    Returns:
        None
    """

    # Check if the CSV file exists
    if not os.path.exists(output_csv):
        # Create a new CSV file with appropriate headers
        pd.DataFrame(columns=[
            "video_name", "frame_count", "anomaly_detected",
            "anomaly_scores", "average_score", "error_message"
        ]).to_csv(output_csv, index=False)

    # Load existing results to avoid duplicate processing
    existing_results = pd.read_csv(output_csv)
    processed_videos = existing_results["video_name"].tolist()

    # Initialize evaluation results
    results = []

    # Iterate through each video file in the directory
    for video_file in Path(video_dir).glob("*.mp4"):
        if video_file.name in processed_videos:
            print(f"Skipping already processed video: {video_file.name}")
            continue

        try:
            print(f"Processing video: {video_file.name}")

            # Step 1: Extract frames
            print("Extracting frames .................................................................\n")
            video_frames_dir, chunk_count = extract_frames(video_path)

            # Step 2: Generate
            print("Generating captions....................................................................\n")
            captions = generate_captions(Path(video_frames_dir))

            # Step 3: Create and refine captions using FAISS
            print("cleaning caption .....................................................................\n")
            index, raw_captions = create_faiss_index(captions)
            refined_captions = refine_captions(index, raw_captions)

            # Step 4: Assign anomaly scores
            print("Creating scores.......................................................................\n")
            scores = anomaly_score(refined_captions)

            # Step 5: Determine if the video contains anomalies
            print("Detecting anomalies....................................................................\n")
            anomaly_detected = is_anomoly(scores, threshold)

            # Log the result
            results.append({
                "video_name": video_file.name,
                "frame_count": frame_count,
                "anomaly_detected": anomaly_detected,
                "anomaly_scores": scores,
                "average_score": sum(scores) / len(scores) if scores else 0,
                "error_message": ""
            })

        except Exception as e:
            print(f"Error processing video {video_file.name}: {e}")
            results.append({
                "video_name": video_file.name,
                "frame_count": 0,
                "anomaly_detected": "Error",
                "anomaly_scores": [],
                "average_score": 0,
                "error_message": str(e)
            })

    # Append new results to the CSV
    if results:
        new_results_df = pd.DataFrame(results)
        new_results_df.to_csv(output_csv, mode='a', header=False, index=False)
        print(f"Updated evaluation results saved to {output_csv}")
    else:
        print("No new videos processed.")


In [18]:
import pandas as pd
import os
video_directory = "./video"
run_eval(video_directory, output_csv="evaluation_results.csv", threshold=0.7)

Processing video: test.mp4
Extracting frames .................................................................

Extracted 13 frames from ./video/test.mp4 to test
Generating captions....................................................................

Error processing video test.mp4: apply_chat_template requires jinja2>=3.1.0 to be installed. Your version is 3.0.3.
Updated evaluation results saved to evaluation_results.csv


In [20]:
%pip uninstall jinja2>=3.1.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate flash_attn

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers (to revision 21fac7abba2a37fae86106f87fcf9974fd1e3830) to /tmp/pip-req-build-o429x9nw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-o429x9nw
  Running command git rev-parse -q --verify 'sha^21fac7abba2a37fae86106f87fcf9974fd1e3830'
  Running command git fetch -q https://github.com/huggingface/transformers 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Running command git checkout -q 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Resolved https://github.com/huggingface/transformers to commit 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=t